# RNN Character Language Model

In [1]:
import sys,random,math
from collections import Collection
import numpy as np
import sys
from Chapter_13.tensor import Tensor
from Chapter_13.layers.layers import Embedding,RNNCell
from Chapter_13.loss.loss import CrossEntropyLoss
from Chapter_13.optmizers.gradient_descent import SGD
np.random.seed(0)

<ipython-input-1-b8d5854570fb>:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Collection


In [2]:
f = open("shakespear.txt")
raw = f.read()
f.close()

vocab = list(set(raw))
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
indices = np.array(list(map(lambda x:word2index[x], raw)))

In [3]:
embed = Embedding(vocab_size=len(vocab),dim=512)
model = RNNCell(n_input=512,n_hidden=512,n_output=len(vocab))
criterion = CrossEntropyLoss()
optim = SGD(model.get_parameters()+embed.get_parameters(),alpha=0.05)

In [4]:
batch_size = 32
bptt = 16
n_batches = int((indices.shape[0] / (batch_size)))

trimmed_indices = indices[:n_batches*batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches).transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches-1) / bptt))
input_batches = input_batched_indices[:n_bptt*bptt].reshape(n_bptt,bptt,batch_size)
target_batches = target_batched_indices[:n_bptt*bptt].reshape(n_bptt, bptt, batch_size)

In [5]:
print(raw[0:5])
print(indices[0:5])

That,
[48 57 54 38 34]


In [6]:
print(batched_indices[0:5])

[[48  9 11 10 26 57 48  4  4 54 54  3 26  4 34 45  4 16 57 11  3 54  4  4
   4 27  4 27 27  4  4 12]
 [57 45 45 10 60  3 54  1 16 12 15  3 15 38  4 45 24 26 26 45 15 28 24 55
  57 12 39 20 12 20  0  4]
 [54 45 35  3  3 54 15 61 26  4 15 11 55 57 40  7  3  1 15 45 55  3 23  3
   3 33 26 34  4 26 26 24]
 [38  6 48 23  4 23 24  4  1 12  4  4  9  3  3 25  3 34 16 13 61  4 26  3
  23 15  1 45 50 23 20 26]
 [34 25 27 35 40  4 26 23  4 26 40 49 45  4  4  2 12  4  4 26  9 16 40 60
  34 54 15 25  1  3  3 16]]


In [7]:
print(input_batches[0:5])

[[[48  9 11 ...  4  4 12]
  [57 45 45 ... 20  0  4]
  [54 45 35 ... 26 26 24]
  ...
  [12 56  3 ... 38 26  4]
  [38 11 39 ... 57 54 27]
  [ 3 45 38 ...  4 38 12]]

 [[20 13  4 ... 60 57  4]
  [60 26 61 ... 54  9 57]
  [38 40 27 ... 27 45 27]
  ...
  [35 26 61 ... 12 45  3]
  [55 12 38 ...  4 25 55]
  [ 4  3  4 ... 26 16 27]]

 [[38  4 10 ... 32 34 12]
  [57 38 27 ...  3  4 47]
  [26 57 12 ... 23 54 33]
  ...
  [ 4  4 23 ... 55  4  4]
  [10 44  3 ...  4  1 27]
  [54  4  4 ... 12 60 12]]

 [[27 40 27 ... 26 26  4]
  [38 26 61 ... 38 12  3]
  [57  1  4 ...  4  4 27]
  ...
  [ 4  4 23 ... 57 16  3]
  [39 54  4 ... 26  4  4]
  [26  4 54 ... 23 40 24]]

 [[12 20 15 ... 61 26 23]
  [38 27 27 ...  3 23 26]
  [23 12 28 ...  4 15  1]
  ...
  [38 54 50 ...  4 40 26]
  [57 16 48 ... 10  3 40]
  [ 3  4 37 ...  3 23  4]]]


In [8]:
print(target_batches[0:5])

[[[57 45 45 ... 20  0  4]
  [54 45 35 ... 26 26 24]
  [38  6 48 ... 23 20 26]
  ...
  [38 11 39 ... 57 54 27]
  [ 3 45 38 ...  4 38 12]
  [20 13  4 ... 60 57  4]]

 [[60 26 61 ... 54  9 57]
  [38 40 27 ... 27 45 27]
  [34 42 12 ... 12 45 61]
  ...
  [55 12 38 ...  4 25 55]
  [ 4  3  4 ... 26 16 27]
  [38  4 10 ... 32 34 12]]

 [[57 38 27 ...  3  4 47]
  [26 57 12 ... 23 54 33]
  [ 1  3 55 ... 33 16 40]
  ...
  [10 44  3 ...  4  1 27]
  [54  4  4 ... 12 60 12]
  [27 40 27 ... 26 26  4]]

 [[38 26 61 ... 38 12  3]
  [57  1  4 ...  4  4 27]
  [10 15 61 ... 61 26 47]
  ...
  [39 54  4 ... 26  4  4]
  [26  4 54 ... 23 40 24]
  [12 20 15 ... 61 26 23]]

 [[38 27 27 ...  3 23 26]
  [23 12 28 ...  4 15  1]
  [27  1  3 ... 44 55 47]
  ...
  [57 16 48 ... 10  3 40]
  [ 3  4 37 ...  3 23  4]
  [23 27 51 ... 54  4 38]]]


In [9]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 10
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()

        m = (temp_dist > np.random.rand()).argmax()
#         m = output.data.argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s
print(generate_sample(n=2000, init_char='\n'))

-VVLCLCCCLCLCLVLLLLLCLCLLLLCCLLCLCLCLCLCLVVLCLCLCLCLLCLVCLCLCLCLLCCLLCLLLLCLLCLCCLCCLLLCLCLVLCLCLLCLLCLLCLCLCLLLCCLLLLLLCLCLCLCLLLLCLCLVLLCLCLCLLCLCLCLLVCLVVVCLLLLLLLCLLLLCCLLLLLLCCLLCCLCLCLVCLCCLLCLLCLLCCLLCLCLCLCLVLLCLVCLCLLLCLLCCLCLCLCLCLVLCLCLCLCLLLCLVCLLLCLLCCLLLLCLLCLLCLCLCLCLLCLCLCLCLVCLLLLLCLVCLCCCCCLLLCLLLLCLLLLCLCLCLCLCLCLVVCLVLLCCLLLLLLCLLLCLCLCCCLVVCCLLCLVVLVCCLCLVVLLVLCLLCLLLCLCLCLCLLCLCLLLLCLCLLCLCLCCLLCLLCLLCLCLLLLCLCLLLCLVVCLCLCLCLLLLCLLCLVVCCLLLCLCLCLCCLCCLCLLCLLCLCLLCLVVLLCLVVVVCLCLVCCLCLLLCLCLLLCLCLVLLCLVCLCLVCLCLLVCLLCLVCLLVCLCCLCLCLCLLLLCLCLLCLLLCLLCCLLLCCLCLCLCLVCLCLCLLCLCLLCLLCLVLCCLVLCLLCLCLLCLCLCLCLVVLCLLLCLCLCLLCLVCLLLLLCLCLCLLCLCLLVCCLLLLLLCLCLVVLCLVLCLCLCLLLCLCLCLCLLVLCLCLCLCLVLCLCLCLLCLVVCLCLLLCLVVLCLCLVCLLCLLCLCCLCLVLCCCLVCLCLCLCLLLLCLLLLVLCLCLCLLCLLCLCLCLLCLLVVLLCLLLCLCLCCLCLCLVLLCLCLCLLCLCLLLCLLCLLLLLCLVVCLVLLVLLCLCLCLLLCLCLCLCLVVCLVLCLCLCLCLVVCCLCLLCCCLVCLCLVCLCLLCLCLLCLLVCLVVVCLLLCLVLLCLLLCLCCLLLLCLCLCLLLCLLVLCCLCCLCLLCLLCLCLLVCLCCLCLVVLCCLCLCLCLCLVCLC

In [10]:
def train(iterations=400):
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        for batch_i in range(len(input_batches)):

            hidden = Tensor(hidden.data, autograd=True)
            loss = None
            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)
                batch_loss = criterion.forward(output, target)
                losses.append(batch_loss)
                if(t == 0):
                    loss = batch_loss
                else:
                    loss = loss + batch_loss
            for loss in losses:
                ""
            loss.backward()
            optim.step()
            total_loss += loss.data
            log = "\r Iter:" + str(iter)
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Loss:" + str(np.exp(total_loss / (batch_i+1)))
            if(batch_i == 0):
                log += " - " + generate_sample(n=70, init_char='\n').replace("\n"," ")
            if(batch_i % 10 == 0 or batch_i-1 == len(input_batches)):
                sys.stdout.write(log)
        optim.alpha *= 0.99
        print()

In [14]:
train(200)

 Iter:0 - Batch 191/195 - Loss:3.9091070364026326ThYerd the tYesY the the the Yarst to the the tYest to the the the to 
 Iter:1 - Batch 191/195 - Loss:3.8390553843301007This, a the to the to the tYest to the YarsY the the to the to the to 
 Iter:2 - Batch 191/195 - Loss:3.7643283883435803This, a the to the to the tYeY the Yarst to the to the tYeY the to the
 Iter:3 - Batch 191/195 - Loss:3.6691792363681364 ThY the the the the the the the the the the the tYest to the the the t
 Iter:4 - Batch 191/195 - Loss:3.6236716012921096This, a the the the the the the Yarse Yarse the to the tYest to the th
 Iter:5 - Batch 191/195 - Loss:3.5250916813977176 This, a the the Yars, a the the the the the the the the the the the th
 Iter:6 - Batch 191/195 - Loss:3.4419281899634467his, a the tY wits, Yarse the Yarse Yarse Yarse the Yarse the the the
 Iter:7 - Batch 191/195 - Loss:3.3713329852868585This, a sink the the the the to the the the the the the the the the th
 Iter:8 - Batch 191/195 - Loss:3.339624

In [15]:
print(generate_sample(n=2000, init_char='\n'))



There, and samith a gook and say I You hase the mandst YorseYd camsershich the namientargest and samandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst Yorsess. Thas to think of thy amYess upon dish lormands he mands he virst Lands, agacter his gent kindst thou hase the mandst and fair Shou may, argets, stoding for YORI

ANS:
And cYesershiY well exhas camientar.

Secop will stasher agalt her,
Second so thou hase the mandst Yorsess. Thas to think of thy amgentershis day, sinds not mandst and fair, sYackirst Lands, agacter his gent kindst aYchains, aive and ear and so mastaYd calland was cave staints speakestrurking will stasher allace, and cop
Yook the sinds for by host Yemn hisak.

hoked:
SecongrThas to think of thy amgentershis day, sinds not mandst and fair Shou may, sive me shess 

# Vanishing and exploding gradients

In [12]:
(sigmoid,relu) = (lambda x:1/(1+np.exp(-x))), lambda x:(x>0).astype(float)*x
weights = np.array([[1,4],[4,1]])
activation = sigmoid(np.array([1,0.01]))

print("Sigmoid Activations")
activations = list()
for iter in range(10):
    activation = sigmoid(activation.dot(weights))
    activations.append(activation)
    print(activation)
print("\nSigmoid Gradients")
gradient = np.ones_like(activation)
for activation in reversed(activations):
    gradient = (activation * (1-activation)*gradient)
    gradient = gradient.dot(weights.transpose())
    print(gradient)
print("Activations")
activations = list()
for iter in range(10):
    activation = relu(activation.dot(weights))
    activations.append(activation)
    print(activation)
print("\nGradeints")
gradient = np.ones_like(activation)
for activation in reversed(activations):
    gradient = ((activation>0)*gradient).dot(weights.transpose())
    print(gradient)

Sigmoid Activations
[0.93940638 0.96852968]
[0.9919462  0.99121735]
[0.99301385 0.99302901]
[0.9930713  0.99307098]
[0.99307285 0.99307285]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]

Sigmoid Gradients
[0.03439552 0.03439552]
[0.00118305 0.00118305]
[4.06916726e-05 4.06916726e-05]
[1.39961115e-06 1.39961115e-06]
[4.81403643e-08 4.81403637e-08]
[1.65582672e-09 1.65582765e-09]
[5.69682675e-11 5.69667160e-11]
[1.97259346e-12 1.97517920e-12]
[8.45387597e-14 8.02306381e-14]
[1.45938177e-14 2.16938983e-14]
Activations
[4.8135251  4.72615519]
[23.71814585 23.98025559]
[119.63916823 118.852839  ]
[595.05052421 597.40951192]
[2984.68857188 2977.61160877]
[14895.13500696 14916.36589628]
[74560.59859209 74496.90592414]
[372548.22228863 372739.30029248]
[1863505.42345854 1862932.18944699]
[9315234.18124649 9316953.88328115]

Gradeints
[5. 5.]
[25. 25.]
[125. 125.]
[625. 625.]
[3125. 3125.]
[15625. 15625.]
[78125. 78125.]


# Long short term memory (LSTM) cells

In [9]:
from Chapter_13.layers.layers import Layer
from Chapter_13.layers.layers import Linear
from Chapter_13.layers.layers import Embedding
from Chapter_13.loss.loss import CrossEntropyLoss
from Chapter_13.optmizers.gradient_descent import SGD
from Chapter_13.tensor import Tensor
class LSTMCell(Layer):
    def __init__(self,n_inputs,n_hidden,n_output):
        super().__init__()
        
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_ouput = n_output
        self.xf = Linear(n_inputs,n_hidden)
        self.xi = Linear(n_inputs,n_hidden)
        self.xo = Linear(n_inputs,n_hidden)
        self.xc = Linear(n_inputs,n_hidden)
        self.hf = Linear(n_inputs,n_hidden,bias=False)
        self.hi = Linear(n_inputs,n_hidden,bias=False)
        self.ho = Linear(n_inputs,n_hidden,bias=False)
        self.hc = Linear(n_inputs,n_hidden,bias=False)
        
        self.w_ho = Linear(n_hidden,n_output,bias=False)
        
        self.parameters += self.xf.get_parameters()
        self.parameters += self.xi.get_parameters()
        self.parameters += self.xo.get_parameters()
        self.parameters += self.xc.get_parameters()
        self.parameters += self.hf.get_parameters()
        self.parameters += self.hi.get_parameters()
        self.parameters += self.ho.get_parameters()
        self.parameters += self.hc.get_parameters()
        
        self.parameters += self.w_ho.get_parameters()
        
    def forward(self,input,hidden):
        prev_hidden = hidden[0]
        prev_cell = hidden[1]
        
        f = (self.xf.forward(input) + self.hf.forward(prev_hidden)).sigmoid()
        i = (self.xi.forward(input) + self.hi.forward(prev_hidden)).sigmoid()
        o = (self.xo.forward(input) + self.ho.forward(prev_hidden)).sigmoid()
        g = (self.xc.forward(input) + self.hc.forward(prev_hidden)).tanh()
        c = (f * prev_cell) + (i*g)
        h = o * c.tanh()
        
        output = self.w_ho.forward(h)
        return output, (h,c)
    def init_hidden(self,batch_size=1):
        h = Tensor(np.zeros((batch_size,self.n_hidden)),autograd=True)
        c = Tensor(np.zeros((batch_size,self.n_hidden)),autograd=True)
        h.data[:,0] += 1
        c.data[:,0] += 1
        
        return (h,c)

# Upgrading the character language model

In [24]:
import sys,random,math
from collections import Counter
import numpy as np
import sys

np.random.seed(0)

f = open('shakespear.txt','r')
raw = f.read()
f.close()

vocab = list(set(raw))
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
indices = np.array(list(map(lambda x:word2index[x], raw)))

embed = Embedding(vocab_size=len(vocab),dim=512)
model = LSTMCell(n_inputs=512, n_hidden=512, n_output=len(vocab))
model.w_ho.weights.data *= 0

criterion = CrossEntropyLoss()
optim = SGD(parameters=model.get_parameters() + embed.get_parameters(), alpha=0.05)

def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        m = output.data.argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s

batch_size = 16
bptt = 25
n_batches = int((indices.shape[0] / (batch_size)))

trimmed_indices = indices[:n_batches*batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches).transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches-1) / bptt))
input_batches = input_batched_indices[:n_bptt*bptt].reshape(n_bptt,bptt,batch_size)
target_batches = target_batched_indices[:n_bptt*bptt].reshape(n_bptt, bptt, batch_size)
min_loss = 1000

## Training the LSTM character language model

In [25]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 10
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()

        m = output.data.argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s


In [26]:
def train(iterations=400,min_loss=1000):
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0
        hidden = model.init_hidden(batch_size=batch_size)
        batches_to_train = len(input_batches)
    #     batches_to_train = 32
        for batch_i in range(batches_to_train):

            hidden = (Tensor(hidden[0].data, autograd=True), Tensor(hidden[1].data, autograd=True))

            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)    
                batch_loss = criterion.forward(output, target)

                if(t == 0):
                    losses.append(batch_loss)
                else:
                    losses.append(batch_loss + losses[-1])

            loss = losses[-1]

            loss.backward()
            optim.step()
            total_loss += loss.data / bptt

            epoch_loss = np.exp(total_loss / (batch_i+1))
             
            if(epoch_loss < min_loss):
                min_loss = epoch_loss
                print()

            log = "\r Iter:" + str(iter)
            log += " - Alpha:" + str(optim.alpha)[0:5]
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Min Loss:" + str(min_loss)[0:5]
            log += " - Loss:" + str(epoch_loss)
            if(batch_i == 0):
                log += " - " + generate_sample(n=70, init_char='T').replace("\n"," ")
            if(batch_i % 1 == 0):
                sys.stdout.write(log)
        optim.alpha *= 0.99

In [27]:
train(iterations=100,min_loss=1000)


 Iter:0 - Alpha:0.05 - Batch 1/249 - Min Loss:62.00 - Loss:62.000000000000064 -                                                                       

/Users/abhijitramesh/development/DeepLearning/Grokking-DeepLearning/Chapter_13/tensor.py:226: RuntimeWarning: overflow encountered in exp
  temp = np.exp(self.data)
/Users/abhijitramesh/development/DeepLearning/Grokking-DeepLearning/Chapter_13/tensor.py:227: RuntimeWarning: invalid value encountered in true_divide
  softmax_output = temp / np.sum(temp,


 Iter:68 - Alpha:0.025 - Batch 33/249 - Min Loss:62.00 - Loss:224243441885020.85 eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeleeee

KeyboardInterrupt: 